

**THIS IS NOT THE COMPLETE TUTORIAL - see file with (MAIN) in the name. Paste all this code before the first Python block**

First you'll need to select which hardware setup you have. You'll need to select both a `SCOPETYPE` and a `PLATFORM`. `SCOPETYPE` can either be `'OPENADC'` for the CWLite/CW1200 or `'CWNANO'` for the CWNano. `PLATFORM` is the target device, with `'CWLITEARM'`/`'CW308_STM32F3'` being the best supported option, followed by `'CWLITEXMEGA'`/`'CW308_XMEGA'`, then by `'CWNANO'`. As of CW 5.4, you can select the SimpleSerial version
used. For example:

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
SS_VER = 'SS_VER_2_1'
```

In [14]:
scope.dis()
target.dis()

In [15]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWHUSKY'
SS_VER = 'SS_VER_2_1'

This code will connect the scope and do some basic setup. We're now just going to use a special setup script to do this. This script contains the commands we ran seperately before.

In [16]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

USBErrorAccess: LIBUSB_ERROR_ACCESS [-3]

USBErrorAccess: LIBUSB_ERROR_ACCESS [-3]

The following code will build the firmware for the target.

In [10]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../firmware/mcu/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2 -j

SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
arm-none-eabi-gcc (GNU Arm Embedded Toolchain 10.3-2021.10) 10.3.1 20210824 (release)
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

mkdir -p objdir-CWHUSKY 
.
Welcome to another exciting ChipWhisperer target build!!
.
.
.
.
.
Compiling:
Compiling:
Compiling:
Compiling:
-en     .././simpleserial/simpleserial.c ...
-en     .././hal/hal.c ...
-en     .././hal//sam4s/startup_sam4s.c ...
Compiling:
-en     basic-passwdcheck.c ...
-en     .././hal//sam4s/sam4s_hal.c ...
.
.
.
.
Compiling:
Compiling:
Compiling:
Compiling:
-en     .././hal//sam4s/uart.c ...
-en     .././hal//sam4s/pio.c ...
-en     .././hal//sam4s/system_sam4s.c ...
-en     .././hal//sam4s/sysclk.c ...
.
Compiling:
-en     .././hal//sam4s/pmc.c ...
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Done!
-e Do

Finally, all that's left is to program the device, which can be done with the following line:

In [12]:
cw.program_target(scope, prog, "../../../firmware/mcu/basic-passwdcheck/basic-passwdcheck-{}.hex".format(PLATFORM))

USBErrorNoDevice: LIBUSB_ERROR_NO_DEVICE [-4]

To make interacting with the hardware easier, let's define a function to attempt a password and return a power trace:

In [13]:
target = cw.target(scope, cw.targets.SimpleSerial2) #cw.targets.SimpleSerial can be omitted

def cap_pass_trace(pass_guess):
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

USBErrorNoDevice: LIBUSB_ERROR_NO_DEVICE [-4]

We also don't need all of the default 5000 samples in the trace. 3000 is a good starting point for most targets:

In [9]:
scope.adc.samples = 3000

USBErrorNoDevice: LIBUSB_ERROR_NO_DEVICE [-4]